<a href="https://colab.research.google.com/github/shaina-12/Artificial-Intelligence/blob/main/Deep%20Learning/Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download the training data from open datasets.
training_data = datasets.KMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)
# Download the test data from the open datasets.
test_data = datasets.KMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/18165135 [00:00<?, ?it/s]

Extracting data/KMNIST/raw/train-images-idx3-ubyte.gz to data/KMNIST/raw



  0%|          | 0/29497 [00:00<?, ?it/s]

Extracting data/KMNIST/raw/train-labels-idx1-ubyte.gz to data/KMNIST/raw



  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting data/KMNIST/raw/t10k-images-idx3-ubyte.gz to data/KMNIST/raw



  0%|          | 0/5120 [00:00<?, ?it/s]

Extracting data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to data/KMNIST/raw



In [3]:
batch_size = 100
# create data loaders.
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([100, 1, 28, 28])
Shape of y: torch.Size([100]) torch.int64


In [4]:
# get the CPU and GPU device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


In [5]:
class MLP(nn.Module):
  def __init__(self,in_dim,out_dim):
    super(MLP,self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(in_dim,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,out_dim),
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [6]:
in_dim=28*28
out_dim=49
model = MLP(in_dim,out_dim).to(device)
print(model)

MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=49, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [8]:
def train(dataloader,model,loss_fn,optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    # compute the prediction error
    pred = model(X)
    loss = loss_fn(pred,y)
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 100 == 0:
      loss, current = loss.item(), batch*len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader,model,loss_fn):
  size =  len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X,y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return 100*correct

In [10]:
epochs = 100
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader,model,loss_fn,optimizer)
  test(test_dataloader,model,loss_fn)

Epoch 1
-------------------------------
loss: 3.885917  [    0/60000]
loss: 3.852851  [10000/60000]
loss: 3.823483  [20000/60000]
loss: 3.776931  [30000/60000]
loss: 3.731881  [40000/60000]
loss: 3.689914  [50000/60000]
Test Error: 
 Accuracy: 20.4%, Avg loss: 3.645472 

Epoch 2
-------------------------------
loss: 3.630900  [    0/60000]
loss: 3.566748  [10000/60000]
loss: 3.487729  [20000/60000]
loss: 3.345800  [30000/60000]
loss: 3.185220  [40000/60000]
loss: 3.057826  [50000/60000]
Test Error: 
 Accuracy: 13.5%, Avg loss: 2.927885 

Epoch 3
-------------------------------
loss: 2.888436  [    0/60000]
loss: 2.762625  [10000/60000]
loss: 2.634164  [20000/60000]
loss: 2.490679  [30000/60000]
loss: 2.375485  [40000/60000]
loss: 2.349434  [50000/60000]
Test Error: 
 Accuracy: 28.1%, Avg loss: 2.358553 

Epoch 4
-------------------------------
loss: 2.292888  [    0/60000]
loss: 2.257750  [10000/60000]
loss: 2.230750  [20000/60000]
loss: 2.190194  [30000/60000]
loss: 2.132238  [40000/6

Note: So, final accuracy of the model will be 76.4% and average loss is 0.759492. On running the model for 1000 epochs, the accuracy of the model will be 89.5%. Due to some technical problem in Google Colab, we cannot able to train the model for 1000 epochs. So, this notebook will be updated soon.